In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2, lognorm, chisquare

In [2]:
df = pd.read_csv("../results/simulation_results_system_4_1.csv", sep=r",(?![^\[]*\])")

/var/folders/3j/3p_dzlvs1nsdfryf9hm7gjnr0000gn/T/ipykernel_6938/3683106496.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("../results/simulation_results_system_4_1.csv", sep=r",(?![^\[]*\])")


In [3]:
df.columns = df.columns.str.lstrip()
df.columns

Index(['Patient ID', 'CTAS Level', 'Tests', 'Arrival Time', 'Departure Time',
       'LOS', 'Triage Waiting Time', 'ED Waiting Time',
       'Medication Waiting Time', 'Inpatient Waiting Time',
       'Triage Waiting Room Length', 'ED Waiting Room Length',
       'Medication Waiting Room Length', 'Inpatient Waiting Room Length'],
      dtype='object')

In [4]:
los_vals = []

for items in df["LOS"]:
    if items > 0:
        los_vals.append(items/60)
        
print(len(los_vals))

98760


In [5]:
los_vals.sort()

In [6]:
# Defining bins and calculating observed frequency
bins = 10
observed_freq, bin_edges = np.histogram(los_vals, bins="auto")
# print(observed_freq, bin_edges)

In [7]:
shape, loc, scale = lognorm.fit(los_vals, floc=0)
print(shape, loc, scale)

0.6854824037288475 0.0 1.1092864537801737


In [8]:
# Calulating expected frequency 
expected_freq = lognorm.pdf(np.arange(1, len(observed_freq)+1), s=shape, loc=loc, scale=scale)
expected_freq *= sum(observed_freq) / sum(expected_freq)

In [9]:
# Calculating Chi-square avlue
chi_sq = sum((observed_freq - expected_freq)**2 / expected_freq)
print(chi_sq)

120930886220.26561


In [10]:
alpha = 0.05
df = len(observed_freq) - 3    # 3 parameters for shape, loc, and scale
crit_val = chi2.ppf(1 - 0.05, df)
print(crit_val)

112.02198574980785


In [11]:
# Compare the test statistic with the critical value

# print("Chi Square value: ", chi_sq, "\nCritical value: ", crit_val)
if chi_sq > crit_val:
# if p < alpha:
    print("Rejects the null hypothesis")
else:
    print("Fails to reject the null hypothesis")

Rejects the null hypothesis
